# Integrated Gradient
https://towardsdatascience.com/interpreting-the-prediction-of-bert-model-for-text-classification-5ab09f8ef074

In [1]:
import torch
import torch.nn as nn
import pandas as pd
from transformers import AutoTokenizer,BertModel
from transformers import EarlyStoppingCallback

In [2]:
model_version = None
tokenizer = AutoTokenizer.from_pretrained(model_version)
bert = BertModel.from_pretrained(model_version)
df = pd.read_csv('../data/BERT/articles_2015_2019_nd_sample.csv')

Some weights of the model checkpoint at CVs/full/bert-base-uncased/fold_full-bert-base-uncased/ were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# Class of model architecture
class BertClassifier(nn.Module):

    def __init__(self):

        super(BertClassifier, self).__init__()

        self.bert = bert
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask = None):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(pooled_output)
        final_layer = self.sigmoid(linear_output)

        return final_layer

### Captum

In [4]:
from captum.attr import LayerIntegratedGradients
from captum.attr import visualization as viz

model = BertClassifier()

# Define model output
def model_output(inputs):
    return model(inputs)[0]

# Define model input
model_input = model.bert.embeddings
lig = LayerIntegratedGradients(model_output, model_input)

In [5]:
def construct_input_and_baseline(text):

    max_length = 510
    baseline_token_id = tokenizer.pad_token_id 
    sep_token_id = tokenizer.sep_token_id 
    cls_token_id = tokenizer.cls_token_id 

    text_ids = tokenizer.encode(text, max_length=max_length, truncation=True, add_special_tokens=False)
   
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    token_list = tokenizer.convert_ids_to_tokens(input_ids)

    baseline_input_ids = [cls_token_id] + [baseline_token_id] * len(text_ids) + [sep_token_id]
    return torch.tensor([input_ids], device='cpu'), torch.tensor([baseline_input_ids], device='cpu'), token_list

def summarize_attributions(attributions):

    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    
    return attributions

def interpret_text(text, true_class):
    input_ids, baseline_input_ids, all_tokens = construct_input_and_baseline(text)
    attributions, delta = lig.attribute(
                                        inputs= input_ids,
                                        baselines= baseline_input_ids,
                                        return_convergence_delta=True,
                                        internal_batch_size=1,
                                        n_steps=50
                                    )
    
    attributions_sum = summarize_attributions(attributions)
    score_vis = viz.VisualizationDataRecord(
                                        word_attributions = attributions_sum,
                                        pred_prob = torch.max(model(input_ids)[0]),
                                        pred_class = torch.argmax(model(input_ids)[0]).numpy(),
                                        true_class = true_class,
                                        attr_class = text,
                                        attr_score = attributions_sum.sum(),       
                                        raw_input_ids = all_tokens,
                                        convergence_score = delta)

    viz.visualize_text([score_vis])
    
for text, sentiment in df.values[:10]:
    true_class = 0
    if sentiment == 'positive':
        true_class = 1
    interpret_text(text, true_class)